# 先把训练集70%、验证集15%、测试集15%的标注文件做好，原始音频切割成3s片段，分好样本集合后随机加噪声。

# 3s分帧加窗，直接在原始音频上处理

In [1]:
import librosa
import os
import glob
import numpy as np
from scipy.io import wavfile

In [2]:
frame_duration = 3  # 帧时长为秒
# overlap_rate = 0.5  # 帧与帧之间的重叠率为50%
overlap_rate = 0  # 帧与帧之间的重叠率为0%

dir_path = r"X:\数据集\ShipEar\shipsEar_AUDIOS"

wav_paths = glob.glob(os.path.join(dir_path, '*.wav'))

# 定义加窗函数
def apply_window(frame):
    window = np.hanning(len(frame))
    return frame * window

In [ ]:
for wav_path in wav_paths:
    dir_name, file_name = os.path.split(wav_path)
    file_name_no_ext = os.path.splitext(file_name)[0]

    # 加载音频
    audio, sr = librosa.load(wav_path, sr=52734)

    # 计算帧的长度和帧移
    frame_length = int(frame_duration * sr)
    hop_length = int(frame_length* (1 - overlap_rate))

    # 填充音频
    total_samples = len(audio)
    if total_samples < frame_length:
        pad_len = frame_length - total_samples
    else:
        remainder = (total_samples - frame_length) % hop_length
        pad_len = 0 if remainder == 0 else hop_length - remainder
    
    audio_padded = np.pad(audio, (0, pad_len), mode="constant") if pad_len > 0 else audio

    # 分帧
    frames = librosa.util.frame(audio_padded, frame_length=frame_length, hop_length=hop_length)

    # 对每帧应用加窗
    frames_windowed = np.apply_along_axis(apply_window, 0, frames)

    output_folder = r"X:\数据集\ShipEar\data_preprocessing\7_frame_window_3s_0%_16kHz"

    os.makedirs(output_folder, exist_ok=True)

    # 保存加窗后的每帧
    for i, frame_windowed in enumerate(frames_windowed.T):
        output_path = os.path.join(output_folder, f"{file_name_no_ext}_{i+1}.wav")
        wavfile.write(output_path, sr, frame_windowed)
        print(f"Processed: {output_path}")
print("All files processed.")